In [1]:
import pandas as pd
import os
import random
import json
from ast import literal_eval
import re
from utils import *
from collections import Counter
from collections import defaultdict
from tqdm import notebook

In [2]:
from sentence_transformers import SentenceTransformer, util

# Load a pre-trained sentence-transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

In [3]:
similar_intent_map = {'express concerns about a player from a failed quest team and suggest to not include them in the current team': 'express concerns about a player from a failed quest team and suggest to remove them from current team',
         'question the leader why they selected a particular player for the team': 'question the leader why they selected a particular player for the team',
         'question the leader why they chose a player for the team': 'question the leader why they selected a particular player for the team',
         'question the leader why they chose a specific player for the team': 'question the leader why they selected a particular player for the team',
         'question the leader why they chose a particular player for the team': 'question the leader why they selected a particular player for the team',
         'question team selection': 'question the leader why they selected a particular player for the team',
         'question leaders choice': 'question the leader why they selected a particular player for the team',
         'counter the team proposal because you think a player on the team is evil': 'counter the team proposal citing that you think a player on the team is evil',
         'counter the team proposal because a player on the team is evil': 'counter the team proposal citing that you think a player on the team is evil',
         'stay hidden in discussions and act like a loyal servant to protect yourself': 'stay hidden in discussions and act like a loyal servant to protect yourself',
         'stay hidden in discussions': 'stay hidden in discussions and act like a loyal servant to protect yourself',
         'stay hidden in discussions to protect yourself': 'stay hidden in discussions and act like a loyal servant to protect yourself',
         'stay hidden as loyal servant': 'stay hidden in discussions and act like a loyal servant to protect yourself',
         'support team proposal': 'support team proposal',
         'supporting team proposal': 'support team proposal',
         'support the team proposal': 'support team proposal',
         'question a player on why he didnt vote for the last team': 'question a player on why he didnt vote for the last team',
         'question a player on why they didnt vote for the last team': 'question a player on why he didnt vote for the last team',
         'defend yourself against allegations that you could be evil': 'defend yourself against allegations that you could be evil',
         'support your teammate by expressing that he is good': 'support your teammate by expressing that he is good',
         'question a player about why they changed their statements': 'question a player about why they changed their statements',
         'provide hints to loyal players about evil players': 'provide hints to loyal players about evil players',
         'provide hints to loyal players': 'provide hints to loyal players about evil players',
         'guide team subtly': 'provide hints to loyal players about evil players',
         'provide hints about evil players': 'provide hints to loyal players about evil players',
         'express concerns about including player in the team': 'express concerns about a player from a failed quest team and suggest to remove them from current team',
         'express concerns about a player from a failed quest team and suggest not including them in current team': 'express concerns about a player from a failed quest team and suggest to remove them from current team',
         'express concerns about a player from a failed quest team and suggest not to include them': 'express concerns about a player from a failed quest team and suggest to remove them from current team',
         'express concerns about a player from a failed quest team and suggest to not include them in current team': 'express concerns about a player from a failed quest team and suggest to remove them from current team',
         'express concerns about a player from a failed quest team and suggest to not include them in current team': 'express concerns about a player from a failed quest team and suggest to remove them from current team',
         'express concerns about a player from a failed quest team and suggest not including them in the current team': 'express concerns about a player from a failed quest team and suggest to remove them from current team',
         'express concerns about a player from a failed quest team and suggest not to include them in the current team': 'express concerns about a player from a failed quest team and suggest to remove them from current team',
         'express concerns about a player from a failed quest team and suggest to not include them in the current team': 'express concerns about a player from a failed quest team and suggest to remove them from current team',
         'express concerns about a player from a failed quest team': 'express concerns about a player from a failed quest team and suggest to remove them from current team',
         'express concerns about a player from a failed quest and suggest to not include them in current team': 'express concerns about a player from a failed quest team and suggest to remove them from current team',
         'express concerns about including a player from a failed quest': 'express concerns about a player from a failed quest team and suggest to remove them from current team',
         'pretend to be merlin by providing hints on who is evil': 'pretend to be merlin by providing hints on who is evil',
         'express disagreement and vote disagree only if you are not in the proposed team': 'express disagreement and vote disagree only if you are not in the proposed team',
         'express disagreement and vote disagree if not on team': 'express disagreement and vote disagree only if you are not in the proposed team',
         'express disagreement and vote disagree if not in the team': 'express disagreement and vote disagree only if you are not in the proposed team',
         'express disagreement and vote disagree if not in team': 'express disagreement and vote disagree only if you are not in the proposed team',
         'express disagreement and vote disagree if not on the team': 'express disagreement and vote disagree only if you are not in the proposed team',
         'express disagreement if not in team': 'express disagreement and vote disagree only if you are not in the proposed team',
         'express disagreement and vote disagree if not in proposed team': 'express disagreement and vote disagree only if you are not in the proposed team',
         'express disagreement and vote disagree only if not in team': 'express disagreement and vote disagree only if you are not in the proposed team',
         'express disagreement if not on team': 'express disagreement and vote disagree only if you are not in the proposed team',
         'counter the team proposal because you are good and not part of the team': 'counter the team proposal citing that you are good and not part of the team',
         'express confusion for failure of mission': 'express confusion for failure of mission',
         'express confusion for failure': 'express confusion for failure of mission',
         'express confusion for failed mission': 'express confusion for failure of mission',
         'include loyal players on the quest team as leader': 'include loyal players on the quest team as leader',
         'support loyal players on quest team as leader': 'include loyal players on the quest team as leader',
         'include loyal players on quest team as leader': 'include loyal players on the quest team as leader',
         'include loyal players on quest team': 'include loyal players on the quest team as leader',
         'pretend to have information and act like merlin': 'pretend to have information and act like merlin',
         'pretend to be merlin to mislead': 'pretend to have information and act like merlin',
         'support your teammate to be on the quest team': 'support your teammate to be on the quest team',
         'support teammate to be on the quest team': 'support your teammate to be on the quest team',
         'support teammate on quest team': 'support your teammate to be on the quest team',
         'support teammate to be on quest team': 'support your teammate to be on the quest team',
         'support evil team composition': 'support your teammate to be on the quest team',
         'support teammate on team': 'support your teammate to be on the quest team',
         'cast suspicion on innocent players': 'cast suspicion on innocent players',
         'cast suspicion on loyal players': 'cast suspicion on innocent players',
         'trying to cast suspicion on innocent players': 'cast suspicion on innocent players',
         'support one loyal player': 'support one loyal player',
         'support loyal players': 'support one loyal player',
         'protect loyal players': 'support one loyal player',
         'express trust in loyal players': 'support one loyal player',
         'share concerns about an evil player': 'share concerns about an evil player',
         'express concerns about evil player': 'share concerns about an evil player',
         'express concerns about an evil player': 'share concerns about an evil player',
         'support loyal players of the previous quest team if the mission failed': 'support loyal players of the previous quest team if the mission failed',
         'defend previous quest team members': 'support loyal players of the previous quest team if the mission failed',
         'support loyal players of the previous quest team': 'support loyal players of the previous quest team if the mission failed',
         'support loyal players from previous quest team': 'support loyal players of the previous quest team if the mission failed',
         'defend your teammate against allegations that they could be evil': 'defend your teammate against allegations that they could be evil',
         'defend your teammate against allegations': 'defend your teammate against allegations that they could be evil',
         'defend teammate against allegations': 'defend your teammate against allegations that they could be evil',
         'defend teammate': 'defend your teammate against allegations that they could be evil',
         'defend teammate against suspicion': 'defend your teammate against allegations that they could be evil',
         'question the leader why they didnt put themselves on the quest team': 'question the leader why they didnt put themselves on the quest team',
         'question the leader why they didnt put themselves on the team': 'question the leader why they didnt put themselves on the quest team',
         'strongly convince the team leader to choose you on team': 'strongly convince the team leader to choose you on team',
         'reveal identity by telling others you are servant to persuade other loyal players': 'reveal identity by telling others you are servant to persuade other loyal players',
         'reveal identity as a loyal player': 'reveal identity by telling others you are servant to persuade other loyal players',
         'defend the proposed team composition if it includes loyal players': 'defend the proposed team composition if it includes loyal players',
         'defend proposed team if includes loyal players': 'defend the proposed team composition if it includes loyal players',
         'defend proposed team with loyal players': 'defend the proposed team composition if it includes loyal players',
         'defend the proposed team if it includes loyal players': 'defend the proposed team composition if it includes loyal players',
         'defend the proposed team composition if includes loyal players': 'defend the proposed team composition if it includes loyal players',
         'support loyal players on team': 'defend the proposed team composition if it includes loyal players',
         'support loyal players on the quest team': 'defend the proposed team composition if it includes loyal players',
         'support the leaders decision': 'support team proposal',
         'express you are a loyal player': 'express you are a loyal player',
         'express loyalty': 'express you are a loyal player',
         'expressing loyalty': 'express you are a loyal player',
         'question why a player didnt vote for the last team': 'question a player on why he didnt vote for the last team',
         'support a loyal player': 'support one loyal player',
         'counter the team proposal because i suspect a player on the team is evil': 'counter the team proposal citing that you think a player on the team is evil',
         'counter the team proposal because you think a player on the team is evil': 'counter the team proposal citing that you think a player on the team is evil',
         'express concerns about a player from a failed quest team and suggest not including them in the current team': 'express concerns about a player from a failed quest team and suggest to remove them from current team',
         'strongly convince the team leader to choose you on the team': 'strongly convince the team leader to choose you on team',
         'question the leader why they selected player for the team': 'question the leader why they selected a particular player for the team',
         'express concerns about including player in the quest team': 'express concerns about a player from a failed quest team and suggest to remove them from current team',
         'counter previous players suspicion on me by expressing loyalty': 'defend yourself against allegations that you could be evil',
         'defend yourself against allegations of being evil': 'defend yourself against allegations of being evil',
         'defend yourself against allegations': 'defend yourself against allegations',
         'question player why they didnt include themselves in the proposed team': 'question the leader why they didnt put themselves on the quest team',
         'question player about their sudden change in stance': 'question a player about why they changed their statements',
         'express loyalty to the team': 'express you are a loyal player',
         'question a player on why they changed their statements': 'question a player about why they changed their statements',
        'pretend that you dont have enough information about who is evil right now': 'pretend that you dont have enough information about who is evil right now',
        'express that you dont have any information on whom to put on the team to protect yourself': 'express that you dont have any information on whom to put on the team to protect yourself',
        'express that you dont have enough information about who is evil right now': 'express that you dont have any information on whom to put on the team to protect yourself',
         'reveal identity by telling others you are percival to persuade other loyal players': 'reveal identity by telling others you are percival to persuade other loyal players',
         'reveal identity as percival': 'reveal identity by telling others you are percival to persuade other loyal players',
         'defend the team composition': 'support team proposal',
         'pretend to be percival by telling others you are percival': 'pretend to be percival by telling others you are percival',
         'pretend to be percival': 'pretend to be percival by telling others you are percival',
        'support the proposed team': 'support team proposal',
        'express concerns about a player from a failed quest and suggest to not include them in the current team': 'express concerns about a player from a failed quest team and suggest to remove them from current team',
        'express disagreement if not on the team': 'express disagreement and vote disagree only if you are not in the proposed team',
        'stay hidden as servant': 'stay hidden in discussions and act like a loyal servant to protect yourself',
        'stay hidden in discussions as a loyal servant': 'stay hidden in discussions and act like a loyal servant to protect yourself',
        'express disagreement and vote disagree': 'express disagreement and vote disagree only if you are not in the proposed team',
        'express disagreement and vote disagree if not on the proposed team': 'express disagreement and vote disagree only if you are not in the proposed team',
        'express concerns about including a player from a failed quest team and suggest to not include them in the current team': 'express concerns about a player from a failed quest team and suggest to remove them from current team',
        'express concerns about a player from a failed quest team and suggest not including them': 'express concerns about a player from a failed quest team and suggest to remove them from current team',
        'express disagreement and vote disagree if not in the proposed team': 'express disagreement and vote disagree only if you are not in the proposed team',
        'stay hidden in discussions to protect self': 'stay hidden in discussions and act like a loyal servant to protect yourself',
        'stay hidden': 'stay hidden in discussions and act like a loyal servant to protect yourself',
        'deceive as a loyal player': 'stay hidden in discussions and act like a loyal servant to protect yourself',
        'stay hidden as evil': 'stay hidden in discussions and act like a loyal servant to protect yourself',

}

In [4]:
len(similar_intent_map)

130

In [5]:
uniq_intents = set(similar_intent_map.values())
for intent in uniq_intents:
    if intent not in similar_intent_map:
        similar_intent_map[intent] = intent

In [6]:
len(similar_intent_map)

133

In [7]:
orig_intents = set(similar_intent_map.values())

In [8]:
orig_intents_embs = {}
for intent in orig_intents:
    orig_intents_embs[intent] = model.encode(intent)

In [9]:
def insert_phrase(sentence):
    max_score = 0
    max_score_intent = ''
    for intent in orig_intents:
        # Generate embeddings
        embeddings1 = model.encode(sentence)
        embeddings2 = orig_intents_embs[intent]

        # Compute cosine similarity
        similarity = util.cos_sim(embeddings1, embeddings2)
        if similarity > max_score:
            max_score = similarity
            max_score_intent = intent

    if max_score >= 0.6:
        similar_intent_map[sentence] = max_score_intent
    

In [10]:
intent_stats_dic = {}
def calc_intent_stats(summarized_intents, selected_intents):
    for intent in summarized_intents:
        if intent not in intent_stats_dic:
            intent_stats_dic[intent] = {'count': 0, 'summary': 0, 'selection': 0, 'both': 0}
        intent_stats_dic[intent]['count'] += 1
        intent_stats_dic[intent]['summary'] += 1
        if intent in selected_intents:
            intent_stats_dic[intent]['selection'] += 1
            intent_stats_dic[intent]['both'] += 1

    for intent in selected_intents:
        if intent not in intent_stats_dic:
            intent_stats_dic[intent] = {'count': 0, 'summary': 0, 'selection': 0, 'both': 0}
        if intent not in summarized_intents:
            intent_stats_dic[intent]['count'] += 1
            intent_stats_dic[intent]['selection'] += 1

In [11]:
def get_results(samples_list):
    results = []
    for r in samples_list:
        intents = [remove_non_alphabets(intent) for intent in r[1]]
        intents = [similar_intent_map[intent] for intent in intents if intent in similar_intent_map]
        guessed_intents = [similar_intent_map[remove_non_alphabets(intent)] for intent in r[2] if remove_non_alphabets(intent) in similar_intent_map]
        # guessed_intents = [remove_non_alphabets(intent) for intent in guessed_intents]
        
        results.append({'guessed_intents': guessed_intents, 'intents': intents})

    
    for res in results:
        calc_intent_stats(res['guessed_intents'], res['intents'])

    total_selection_count = 0
    selection_in_summary_count = 0
    total_summary_count = 0
    for key, value in intent_stats_dic.items():
        total_selection_count += value['selection']
        selection_in_summary_count += value['both']
        total_summary_count += value['summary']
    # print(total_selection_count, total_summary_count)
    recall = selection_in_summary_count / total_selection_count
    precision = selection_in_summary_count/total_summary_count
    f1score = calculate_f1_score(recall, precision)
    return f1score, recall, precision

In [1225]:
# ANNOTATED_FILE = "output/gpt-3_5-turbo-games/annotated/evaluation/guessing_annotations/refine_con_intent_following_without_intents_100_guessing_sy (1).csv"
# ANNOTATED_FILE = "output/gpt-3_5-turbo-games/annotated/evaluation/guessing_annotations/refine_con_intent_following_without_intents_100_javin.csv"
# ANNOTATED_FILE = "output/gpt-3_5-turbo-games/annotated/evaluation/guessing_annotations/refine_con_intent_following_without_intents_100_guessing_Final.csv"
# gpt 4
ANNOTATED_FILE = "output/gpt-4-games/annotated/evaluation/guessing_annotations/intent_guessing_gpt4_without_intents_50_Ziyi.csv"
# ANNOTATED_FILE = "output/gpt-4-games/annotated/evaluation/guessing_annotations/intent_guessing_gpt4_without_intents_50_javin.csv"
# ANNOTATED_FILE = "output/gpt-4-games/annotated/evaluation/guessing_annotations/intent_guessing_gpt4_without_intents_50_Abhi.csv"

In [1226]:
guessing_annotated_df = pd.read_csv(ANNOTATED_FILE)
guessing_annotated_df.head()

,id,context,speak,Unnamed: 3
0,game_2024-03-29-12-52-20-player3-4,**Name**: Player4\n\n**Role**: Servant\n\n**Ro...,"Team, our previous quest outcomes must guide o...",Question the leader why they selected a partic...
1,game_2024-03-29-1-59-14-player4-2,**Name**: Player5\n\n**Role**: Assassin\n\n**R...,"Player2, I understand the need for a strong te...",Question the leader why they selected a partic...
2,game_2024-03-29-1-27-24-player5-3,**Name**: Player1\n\n**Role**: Assassin\n\n**R...,"Player3, I find it risky to include a member f...",express disagreement and vote disagree only if...
3,game_2024-03-29-0-16-48-player2-4,**Name**: Player3\n\n**Role**: Assassin\n\n**R...,"While I trust Player4's judgment, the inclusio...",counter the team proposal citing that you thin...
4,game_2024-03-29-1-27-24-player2-3,**Name**: Player3\n\n**Role**: Morgana\n\n**Ro...,Player3's decision to include Player1 again is...,Question the leader why they selected a partic...


In [1227]:
guessing_annotated_df['Unnamed: 3'].values[2]

'express disagreement and vote disagree only if you are not in the proposed team,Question the leader why they selected a particular player for the team'

In [1228]:
# GOLD_FILE = "output/gpt-3_5-turbo-games/annotated/evaluation/structured_context_files/refine_con_intent_following_100.csv"

# gpt 4
GOLD_FILE = "output/gpt-4-games/annotated/evaluation/structured_context_files/intent_guessing_gpt4_50.csv"

In [1229]:
guessing_gold_df = pd.read_csv(GOLD_FILE)
guessing_gold_df.head()

,id,context,intents,speak
0,game_2024-03-29-12-52-20-player3-4,**Name**: Player4\n\n**Role**: Servant\n\n**Ro...,['question a player on why he didnt vote for t...,"Team, our previous quest outcomes must guide o..."
1,game_2024-03-29-1-59-14-player4-2,**Name**: Player5\n\n**Role**: Assassin\n\n**R...,['question the leader why they selected a part...,"Player2, I understand the need for a strong te..."
2,game_2024-03-29-1-27-24-player5-3,**Name**: Player1\n\n**Role**: Assassin\n\n**R...,['question the leader why they selected a part...,"Player3, I find it risky to include a member f..."
3,game_2024-03-29-0-16-48-player2-4,**Name**: Player3\n\n**Role**: Assassin\n\n**R...,['defend yourself against allegations that you...,"While I trust Player4's judgment, the inclusio..."
4,game_2024-03-29-1-27-24-player2-3,**Name**: Player3\n\n**Role**: Morgana\n\n**Ro...,['question the leader why they selected a part...,Player3's decision to include Player1 again is...


In [1230]:
def clean_annotation(annotation):
    if (type(annotation) != str):
        return ''
    split_intents = re.split(r"\d[.]+|\n", annotation)
    guessed_intents = []
    for line in split_intents:
        if len(line) > 10:
            processed_intent = remove_non_alphabets(line.strip())
            guessed_intents.append(processed_intent)

    # summarized_intents = [remove_non_alphabets(intent.split('. ')[-1]) for intent in r[1].split('\n')]
    guessed_intents_final = []
    for intent in guessed_intents:
        split_intents = intent.strip().split("  ")
        split_intents = [i.strip() for i in split_intents]
        guessed_intents_final.extend(split_intents)

    return ','.join(guessed_intents_final)

In [1231]:
def clean_annotation_v2(annotation):
    if (type(annotation) != str):
        return ''
    split_intents = re.split(',', annotation)
    guessed_intents = []
    for line in split_intents:
        if len(line) > 10:
            processed_intent = remove_non_alphabets(line.strip())
            guessed_intents.append(processed_intent)

    # summarized_intents = [remove_non_alphabets(intent.split('. ')[-1]) for intent in r[1].split('\n')]
    # guessed_intents_final = []
    # for intent in guessed_intents:
    #     split_intents = intent.strip().split("  ")
    #     split_intents = [i.strip() for i in split_intents]
    #     guessed_intents_final.extend(split_intents)

    return ','.join(guessed_intents)

In [1232]:
# guessing_annotated_df['guessed_intents'] = guessing_annotated_df['Unnamed: 3'].apply(lambda x: clean_annotation(x))
# gpt 4 javin
guessing_annotated_df['guessed_intents'] = guessing_annotated_df['Unnamed: 3'].apply(lambda x: clean_annotation_v2(x))
guessing_annotated_df.head()

,id,context,speak,Unnamed: 3,guessed_intents
0,game_2024-03-29-12-52-20-player3-4,**Name**: Player4\n\n**Role**: Servant\n\n**Ro...,"Team, our previous quest outcomes must guide o...",Question the leader why they selected a partic...,question the leader why they selected a partic...
1,game_2024-03-29-1-59-14-player4-2,**Name**: Player5\n\n**Role**: Assassin\n\n**R...,"Player2, I understand the need for a strong te...",Question the leader why they selected a partic...,question the leader why they selected a partic...
2,game_2024-03-29-1-27-24-player5-3,**Name**: Player1\n\n**Role**: Assassin\n\n**R...,"Player3, I find it risky to include a member f...",express disagreement and vote disagree only if...,express disagreement and vote disagree only if...
3,game_2024-03-29-0-16-48-player2-4,**Name**: Player3\n\n**Role**: Assassin\n\n**R...,"While I trust Player4's judgment, the inclusio...",counter the team proposal citing that you thin...,counter the team proposal citing that you thin...
4,game_2024-03-29-1-27-24-player2-3,**Name**: Player3\n\n**Role**: Morgana\n\n**Ro...,Player3's decision to include Player1 again is...,Question the leader why they selected a partic...,question the leader why they selected a partic...


In [1233]:
guessing_gold_data = guessing_gold_df[['id', 'intents']].values.tolist()
guessing_gold_data_dic = {key: literal_eval(intents) for key, intents in guessing_gold_data}
len(guessing_gold_data_dic)

50

In [1234]:
guessing_annotated_df['guessed_intents'].values[:5]

array(['question the leader why they selected a particular player for the team,question a player on why he didnt vote for the last team',
       'question the leader why they selected a particular player for the team',
       'express disagreement and vote disagree only if you are not in the proposed team,question the leader why they selected a particular player for the team',
       'counter the team proposal citing that you think a player on the team is evil,defend yourself against allegations that you could be evil,express concerns about a player from a failed quest team and suggest to remove them from current team',
       'question the leader why they selected a particular player for the team,defend the proposed team composition if it includes loyal players'],
      dtype=object)

In [1235]:
guessing_annotated_df = guessing_annotated_df.dropna(subset=["id"])
len(guessing_annotated_df)

50

In [1236]:
annotation_results = guessing_annotated_df[['id', 'guessed_intents']].values.tolist()
annotated_samples = []
missing_count = 0
for r in annotation_results:
    if r[0] not in guessing_gold_data_dic:
        missing_count += 1
        continue
    data = {}
    data['id'] = r[0]
    data['intents'] = guessing_gold_data_dic[r[0]]
    data['guessed_intents'] = r[1].split(',')
    annotated_samples.append(data)

In [1237]:
missing_count

0

In [1238]:
annotated_samples[10]

{'id': 'game_2024-03-29-1-27-24-player4-3',
 'intents': ['question the leader why they selected a particular player for the team',
  'express concerns about a player from a failed quest team and suggest to not include them in current team',
  'counter the team proposal because you think a player on the team is evil'],
 'guessed_intents': ['question the leader why they selected a particular player for the team',
  'express concerns about a player from a failed quest team and suggest to remove them from current team',
  'support loyal players of the previous quest team if the mission failed']}

In [1239]:
results = []
for sample in annotated_samples:
    if 'guessed_intents' not in sample:
        continue
    intents = [remove_non_alphabets(intent) for intent in sample['intents']]
    intents = [similar_intent_map[intent] for intent in intents if intent in similar_intent_map]
    summarized_intents = [remove_non_alphabets(intent) for intent in sample['guessed_intents'] if intent != '']
    
    results.append({'guessed_intents': summarized_intents, 'intents': intents})

In [1240]:
results[45]

{'guessed_intents': ['support team proposal',
  'defend your teammate against allegations that they could be evil'],
 'intents': ['support team proposal',
  'support your teammate by expressing that he is good',
  'stay hidden in discussions and act like a loyal servant to protect yourself']}

In [1241]:
count1, count2 = 0, 0
for r in results:
    count1 += len(r['guessed_intents'])
    count2 += len(r['intents'])
count1, count2

(115, 146)

In [1242]:
results[1]

{'guessed_intents': ['question the leader why they selected a particular player for the team'],
 'intents': ['question the leader why they selected a particular player for the team',
  'counter the team proposal citing that you think a player on the team is evil',
  'express concerns about a player from a failed quest team and suggest to remove them from current team']}

In [1243]:
intent_stats_dic = {}
for res in results:
    calc_intent_stats(res['guessed_intents'], res['intents'])

In [1244]:
total_selection_count = 0
selection_in_summary_count = 0
total_summary_count = 0
for key, value in intent_stats_dic.items():
    total_selection_count += value['selection']
    selection_in_summary_count += value['both']
    total_summary_count += value['summary']

In [1245]:
recall = selection_in_summary_count / total_selection_count
recall

0.6027397260273972

In [1246]:
precision = selection_in_summary_count/total_summary_count
precision

0.7652173913043478

In [1247]:
calculate_f1_score(recall, precision)

0.6743295019157087

### round wise

In [1248]:
samples_list = []
total_sel = 0
total_guessed = 0
for s in annotated_samples:
    if 'guessed_intents' in s:
        total_sel += len(s['intents'])
        total_guessed += len(s['guessed_intents'])
        samples_list.append([s['id'], s['intents'], s['guessed_intents']])

In [1249]:
rounds = defaultdict(list)
for item in samples_list:
    round_no = int(item[0].split('-')[-1])%10
    rounds[round_no].append(item)
len(rounds)

5

In [1250]:
for rnd in sorted(rounds.keys()):
    intent_stats_dic = {}
    f1score, recall, precision = get_results(rounds[rnd])
    f1score, recall, precision = f1score, recall*100, precision*100
    # print(f"Round: {rnd} => {f1score=:.2f} {recall=:.2f} {precision=:.2f}")
    print(f"Round: {rnd} => {f1score=:.2f}")

Round: 1 => f1score=0.67
Round: 2 => f1score=0.64
Round: 3 => f1score=0.67
Round: 4 => f1score=0.78
Round: 5 => f1score=0.50


GPT 3.5 

Deepayan(final file) = f1 = 0.6193, recall = 0.5745, precision = 0.6717

1. Round: 1 => f1score=0.62
2. Round: 2 => f1score=0.61
3. Round: 3 => f1score=0.65
4. Round: 4 => f1score=0.60

Javin = f1 = 0.5772, recall = 0.5, precision = 0.6826

1. Round: 1 => f1score=0.61
2. Round: 2 => f1score=0.54
3. Round: 3 => f1score=0.53
4. Round: 4 => f1score=0.64

Sy = f1 = 0.6439, recall = 0.5789, precision = 0.7252

1. Round: 1 => f1score=0.62
2. Round: 2 => f1score=0.63
3. Round: 3 => f1score=0.68
4. Round: 4 => f1score=0.64

std = 0.0275

Avg = 0.6134

Round 1 = 0.62, 004
Round 2 = 0.59, 0.03
Round 3 = 0.62, 0.06
Round 4 = 0.63, 0.02

Gpt 4 guessing javing results on 50 samples = f1 = 0.5826, recall = 0.4589, precision = 0.7976 <br/>
GPt 4 guessing Ziyi results on 50 samples = f1 = 0.6743, recall = 0.6027, precision = 0.7652<br/>
GPt 4 guessing Abhi results on 50 samples = f1 = 0.70, recall = 0.6712, precision = 0.7313

std = 0.0503

Javin

1. Round: 1 => f1score=0.67
2. Round: 2 => f1score=0.56
3. Round: 3 => f1score=0.55
4. Round: 4 => f1score=0.57
5. Round: 5 => f1score=0.67


Ziyi

1. Round: 1 => f1score=0.67
2. Round: 2 => f1score=0.64
3. Round: 3 => f1score=0.67
4. Round: 4 => f1score=0.78
5. Round: 5 => f1score=0.50

Abhishek

1. Round: 1 => f1score=0.74
2. Round: 2 => f1score=0.75
3. Round: 3 => f1score=0.84
4. Round: 4 => f1score=0.86
5. Round: 5 => f1score=0.62

Avg = 

Round 1 = 0.67, 0.0
Round 2 = 0.60, 0.04
Round 3 = 0.61, 0.06
Round 4 = 0.67, 0.10
Round 5 = 0.58, 0.08

# Model results

In [12]:
# RESULTS_FILE = "output/gpt-4-games/annotated/evaluation/guessing_model_results/intent_guessing_all_reflective_gpt_3_5_results.json"
RESULTS_FILE = "output/gpt-3_5-turbo-games/annotated/evaluation_our_annotations/guessing_model_results/intent_guessing_second_order_5_games_results.json"

In [19]:
FIRST_ORDER_RESULTS_FILE = 'output/gpt-3_5-turbo-games/annotated/evaluation_our_annotations/guessing_model_results/intent_guessing_results.json'

In [13]:
with open(RESULTS_FILE) as f:
    data = json.load(f)
samples = list(data.values())

In [20]:
with open(FIRST_ORDER_RESULTS_FILE) as f:
    first_order_data = json.load(f)

In [22]:
for sample in samples:
    first_order_intents = first_order_data[sample['id']]['guessed_intents']
    sample['first_order_intents'] = first_order_intents

In [23]:
samples[6]

{'id': 'game_2024-03-06-16-53-2-player5-3',
 'context': "**Name**: Player5\n\n**Role**: Merlin\n\n**Role Details**: \nYou are Merlin. These are your goals and characteristics: \n- Know the identities of evil players.\n- Subtly guide your team, especially Percival.\n- Avoid behaviours that expose your role: overly accusing, being too helpful.\n- Goal(Main Intent): Win without revealing identity.\n\n**Round**: 3\n\n**Player Name**: Player4\n\n**Current Leader**: Player3\n\n**Current Team**: Player1, Player2\n\n**Previous Rounds Team Voting**:\nRound 1: Player1 = agree, Player2 = agree, Player3 = disagree, Player4 = agree, Player5 = disagree \nRound 2: Player1 = disagree, Player2 = agree, Player3 = disagree, Player4 = agree, Player5 = agree \nRound 3: Player1 = agree, Player2 = disagree, Player3 = agree, Player4 = agree, Player5 = disagree \n\n**Previous Results**: \nRound 1: Team = Player1, Player2, Result = success\nRound 2: Team = Player1, Player4, Player5, Result = success\n\n**Previo

In [24]:
samples_list = []
total_sel = 0
total_guessed = 0
for s in samples:
    if 'guessed_intents' in s:
        total_sel += len(s['first_order_intents'])
        total_guessed += len(s['guessed_intents'])
        samples_list.append([s['id'], s['first_order_intents'], s['guessed_intents']])

In [25]:
total_sel, total_guessed

(188, 179)

In [26]:
uniq_sum_intents = []
for ann in samples_list:
    for intent in ann[2]:
        intent = remove_non_alphabets(intent)
        uniq_sum_intents.append(intent)

print(len(uniq_sum_intents))
uniq_sum_intents1 = set(uniq_sum_intents)
print(len(uniq_sum_intents1))

count = 0
st_count = 0
rephrased_intents = set()
for intent in uniq_sum_intents:
    if intent not in similar_intent_map:
        count += 1
        rephrased_intents.add(intent)
count

179
45


3

In [27]:
for rephrased_intent in notebook.tqdm(rephrased_intents):
    insert_phrase(rephrased_intent)

  0%|          | 0/3 [00:00<?, ?it/s]

In [28]:
# annotated_samples = []
# missing_count = 0
# for r in samples_list:
#     # if r[0] not in all_data_dic:
#     #     missing_count += 1
#     #     continue
#     data = {}
#     data['intents'] = r[1]
#     # split_intents = r[1].split('\n')
#     data['guessed_intents'] = [similar_intent_map[remove_non_alphabets(intent)] for intent in r[2] if remove_non_alphabets(intent) in similar_intent_map]
#     annotated_samples.append(data)

In [29]:
samples_list[1]

['game_2024-03-06-16-53-2-player2-2',
 ['Support loyal players',
  'Provide hints to loyal players about evil players'],
 ['express concerns about a player from a failed quest team and suggest to not include them in the current team']]

In [30]:
# len(annotated_samples)

In [31]:
def calculate_f1_score(precision, recall):
    if precision + recall == 0:
        return 0.0
    else:
        return 2 * (precision * recall) / (precision + recall)

In [32]:
impactful_intents = {
    'cast suspicion on innocent players': True, 
    'counter the team proposal because you think a player on the team is evil': True, 
    'express concerns about a player from a failed quest team and suggest to not include them in the current team': True, 
    'express confusion for failure of mission': True, 
    'express disagreement and vote disagree only if you are not in the proposed team': True, 
    'express you are a loyal player': True, 
    'pretend to be merlin by providing hints on who is evil': True, 
    'question a player about why they changed their statements': True, 
    'question a player on why they didnt vote for the last team': True, 
    'question the leader why they selected a particular player for the team': True, 
    'reveal identity by telling others you are servant to persuade other loyal players': True, 
    'share concerns about an evil player': True, 
    'strongly convince the team leader to choose you on team': True, 
    'support loyal players of the previous quest team if the mission failed': True, 
    'support your teammate by expressing that he is good': True
}

In [33]:
def get_results(samples_list):
    results = []
    for r in samples_list:
        intents = [remove_non_alphabets(intent) for intent in r[1]]
        intents = [similar_intent_map[intent] for intent in intents if intent in similar_intent_map]
        guessed_intents = [similar_intent_map[remove_non_alphabets(intent)] for intent in r[2] if remove_non_alphabets(intent) in similar_intent_map]
        results.append({'guessed_intents': guessed_intents, 'intents': intents})

    
    for res in results:
        calc_intent_stats(res['guessed_intents'], res['intents'])

    total_selection_count = 0
    selection_in_summary_count = 0
    total_summary_count = 0
    for key, value in intent_stats_dic.items():
        total_selection_count += value['selection']
        selection_in_summary_count += value['both']
        total_summary_count += value['summary']
    print(total_selection_count, total_summary_count)
    recall = selection_in_summary_count / total_selection_count
    precision = selection_in_summary_count/total_summary_count
    f1score = calculate_f1_score(recall, precision)
    return f1score, recall, precision

In [34]:
intent_stats_dic = {}

### Results on complete data

In [35]:
get_results(samples_list)

184 176


(0.05555555555555556, 0.05434782608695652, 0.056818181818181816)

gpt 3.5 results with perspective example - (0.3009523809523809, 0.27147766323024053, 0.33760683760683763) <br/>
gpt 3.5 results with perspective example on gpt 4 games for 50 turns - (0.15702479338842978, 0.1292517006802721, 0.2)

### Updated results on just our annotation's files
gpt 4 vanilla results - (0.2020, 0.1492, 0.3125) <br/>
gpt 4 reflective results - (0.4402, 0.3482, 0.5982) <br/>
gpt 3.5 reflective results on gpt 4 data - (0.2395, 0.1990, 0.3007)

gpt 3.5 vanilla results - (0.0778, 0.0701, 0.0874) <br/>
gpt 3.5 reflective results - (0.2973, 0.2719, 0.3280) <br/>
gpt 4 reflective results on gpt 3.5 data - (0.3491, 0.3070, 0.4046)


gpt 3.5 all vanilla = (0.31632653061224486, 0.27978339350180503, 0.36384976525821594)

100 - (0.38399999999999995, 0.34782608695652173, 0.42857142857142855)

reflective 100 - (0.40108401084010836, 0.357487922705314, 0.4567901234567901)

In [815]:
rounds = defaultdict(list)
for item in samples_list:
    round_no = int(item[0].split('-')[-1])%10
    rounds[round_no].append(item)
len(rounds)

5

### Round wise results

In [816]:
for rnd in sorted(rounds.keys()):
    intent_stats_dic = {}
    f1score, recall, precision = get_results(rounds[rnd])
    f1score, recall, precision = f1score, recall*100, precision*100
    # print(f"Round: {rnd} => {f1score=:.2f} {recall=:.2f} {precision=:.2f}")
    print(f"Round: {rnd} => {f1score=:.2f}")

Round: 1 => f1score=0.25
Round: 2 => f1score=0.36
Round: 3 => f1score=0.35
Round: 4 => f1score=0.37
Round: 5 => f1score=0.32


Round wise results (our annotations)

GPT 3.5 Vanilla -

1. Round: 1 => f1score=0.02
2. Round: 2 => f1score=0.11
3. Round: 3 => f1score=0.05
4. Round: 4 => f1score=0.11

GPT 3.5 Reflective - 

1. Round: 1 => f1score=0.25
2. Round: 2 => f1score=0.34
3. Round: 3 => f1score=0.32
4. Round: 4 => f1score=0.27

GPT 4 on GPT 3.5 data (reflective) - 

1. Round: 1 => f1score=0.32
2. Round: 2 => f1score=0.37
3. Round: 3 => f1score=0.31
4. Round: 4 => f1score=0.39


GPT 4 Vanilla - 

1. Round: 1 => f1score=0.29
2. Round: 2 => f1score=0.18
3. Round: 3 => f1score=0.17
4. Round: 4 => f1score=0.15
5. Round: 5 => f1score=0.23

GPT 4 Reflective -

1. Round: 1 => f1score=0.35
2. Round: 2 => f1score=0.32
3. Round: 3 => f1score=0.47
4. Round: 4 => f1score=0.55
5. Round: 5 => f1score=0.56

GPT 3.5 on GPT 4 data (reflective) - 

1. Round: 1 => f1score=0.07
2. Round: 2 => f1score=0.22
3. Round: 3 => f1score=0.30
4. Round: 4 => f1score=0.34
5. Round: 5 => f1score=0.35